<a href="https://colab.research.google.com/github/Brian-Clough/case_studies/blob/main/mozambique_biomass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Technical challenge: Estimating total biomass for two concessions in Mozambique
Brian J. Clough, PhD

*February 2026*

##Introduction
This notebook contains a walkthrough biomass analysis for two concessions in Mozambique. It accompanies the submitted technical report and outlines a walkthrough of the complete work flow used to generate the results presented therein.

To allow for reproducibility this notebook will present the full workflow, beginning with the raw plot and concession boundary data.
##Analysis overview
The biomass analysis proceeds with the following steps:


*   Read in and clean inventory data, then summarize to plot-level biomass estimates
*   Acquire satellite imagery and post-stratify the concessions
*   Develop and fit a model of total biomass for the concession areas
*   Evaluate model performance
*   Visualize and assess results

First, let's install/load some packages and connect to Google Earth Engine, which we will use to access imagery to perform the post-stratification. Note that to use Google Earth Engine, you need to set up your own Google Cloud Project and update the `project_id` object in the chunk below. However, I have made the stratification layer available for download on the accompanying Github repository, so you can feel free to skip over the image processing steps.


In [43]:
# Package load
import os
import ee
import folium
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import geemap
import xarray as xr
import numpy as np

# Authenticate & connect to Google Earth Engine
project_id = 'resilience-480318'
ee.Authenticate()
ee.Initialize(project=project_id)


Now, let's get the data and open the files in our python session. We are also going to convert the bounds to an EE object for easy rendering/analysis in Google Earth Engine.

In [44]:
!wget -P /tmp/ https://github.com/Brian-Clough/case_studies/raw/main/data/mozambique_bounds.gpkg
!wget -P /tmp/ "https://github.com/Brian-Clough/case_studies/raw/main/data/LF_BIOMASS SURVEY_ITclean-FULL.xlsx"
!wget -P /tmp/ "https://github.com/Brian-Clough/case_studies/raw/main/data/MPM_BIOMASS SURVEY_ITclean-FULL.xlsx"


bounds_gdf = gpd.read_file('/tmp/mozambique_bounds.gpkg', layer='project_mozambique')

#Convert bounds to EE object for visualization
bounds_ee = geemap.gdf_to_ee(bounds_gdf)

--2026-02-23 21:02:56--  https://github.com/Brian-Clough/case_studies/raw/main/data/mozambique_bounds.gpkg
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Brian-Clough/case_studies/main/data/mozambique_bounds.gpkg [following]
--2026-02-23 21:02:56--  https://raw.githubusercontent.com/Brian-Clough/case_studies/main/data/mozambique_bounds.gpkg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176128 (172K) [application/octet-stream]
Saving to: ‘/tmp/mozambique_bounds.gpkg.7’

mozambique_bounds.g 100%[===================>] 172.00K  --.-KB/s    in 0.009s  

2026-02-23 21:02:57 (18.5 MB/s) - ‘/tmp/moza

In [45]:

ee_map = geemap.Map()
ee_map.centerObject(bounds_ee, zoom = 9)
ee_map.add_basemap("HYBRID")
ee_map.add_ee_layer(bounds_ee, {}, name="bounds")
ee_map

Map(center=[-18.887040535302553, 35.0502820401025], controls=(WidgetControl(options=['position', 'transparent_…

Now, we must process the inventory data. This is going to involve several steps:


1.   Parse the spatial coordinates from the survey headers on each plot tab
2.   Parse crosswalk to link local species names to latin names for allometric model selection
3.   Develop clean `tree_data` and `plot_data` objects using these and the consolidated survey data in each workbook.



In [ ]:
#First, let's get the consolidated tree table. Note I spot-checked to ensure that the survey
#data here matches the survey data on each individual plot tab. In a production scenario we would
#write some tests
raw_tree_data = pd.read_excel('/tmp/LF_BIOMASS SURVEY_ITclean-FULL.xlsx', sheet_name='Consolidated', skiprows=4)
raw_tree_data = raw_tree_data.drop(raw_tree_data.columns[0], axis=1) #Drop empty first column

#Rename the columns for easier joining/handling
raw_tree_data = raw_tree_data.rename(columns={
    'Sampling / Survey Date': 'survey_date',
    'Recorder (lead)': 'recorder',
    'Carbon Site ID': 'plot_id',
    'Plot Surface': 'plot_area',
    'Estimated Plot Age': 'plot_age',
    'Tree No.': 'tree_number',
    'Local Name': 'local_name',
    'Stem No.' : 'n_stems',
    'DBH (cm)' : 'dbh_cm',
    'Tree Height (m)' : 'total_height_m'})

#Now, let's get the species list
species_cw = pd.read_excel('/tmp/LF_BIOMASS SURVEY_ITclean-FULL.xlsx', sheet_name='Moz Species List', skiprows=9)
species_cw = species_cw.drop(species_cw.columns[0], axis=1) #Drop empty first column
species_cw = species_cw.rename(columns={
    'Local Name' : 'local_name',
    'Scientific Name' : 'scientific_name'})

#Lastly we want to parse the coordinates from the survey tabs,


     survey_date recorder plot_id  Plot surface (m2) plot_age  tree_number  \
0     2025-11-27       LF   LF_01                196        4          1.0   
1     2025-11-27       LF   LF_01                196        4          2.0   
2     2025-11-27       LF   LF_01                196        4          3.0   
3     2025-11-27       LF   LF_01                196        4          4.0   
4     2025-11-27       LF   LF_01                196        4          5.0   
...          ...      ...     ...                ...      ...          ...   
1177  2025-12-05       LF   LF_23                196        7         17.0   
1178  2025-12-05       LF   LF_23                196        7         17.0   
1179  2025-12-05       LF   LF_23                196        7         17.0   
1180  2025-12-05       LF   LF_23                196        7         17.0   
1181  2025-12-05       LF   LF_23                196        7         17.0   

       local_name  n_stems dbh_cm total_height_m  
0        Nch